In [1]:
import graphlab as gl
from graphlab import SArray
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from pylab import *
from itertools import cycle
from sklearn import metrics

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1490136395.log


This non-commercial license of GraphLab Create for academic use is assigned to victor.li.zhu@rutgers.edu and will expire on October 06, 2017.


In [ ]:
# given the l2-norm penalty weight and feature type, 
# plot the ROC for each window length, and shift each subject for testing data
# average ROC data across subjects for each window length

# input parameters
#=================================
classifier_list = ['KNN']
#=================================

window_length_list = [250] #, 200, 250, 300]
window_step = 50
feature_type_list = [1] #,2,3,12,13,23,123]  # 1: dg, 2: cc, 3: pl, 12: dg+cc, 13: dg+pl, 23: cc+pl, 123: dg+cc+pl
k    = 9

for win_ln_idx in range(len(window_length_list)): # NOTE HERE!!!!! REMOVE "1" AFTER THIS SESSION!!!!!!!!!!!!!
    window_length = window_length_list[win_ln_idx]

    # print on screen: progress
    current_progress = ("\n========================================\nLoading features for window_length%03d ...\n" \
                        % (window_length))
    print current_progress
    
    for feature_type_idx in range(len(feature_type_list)):
        feature_type = feature_type_list[feature_type_idx]
        
        colors = ['steelblue', 'orangered','gold','purple','yellowgreen','deepskyblue','firebrick']

        FP = []
        TP = []
        
        for mouse_id_test in range(1,3):
            
            ##========================= load features
            # load and organize feature matrix
            comb_train, comb_test = lz_load_feature_matrix_across_subject(mouse_id_test, window_length, window_step)   

            results = lz_KNN_across_subjects_AC_SE_SP(comb_train, comb_test, feature_type, k)
                        
            roc = results['roc_curve']
            
            FP.append(roc['fpr'])
            TP.append(roc['tpr'])
            
        FP = np.array(FP)
        TP = np.array(TP)
        
        print ("==========\n")
        print FP.shape
        print ("==========\n")


        FP_mn = np.mean(FP, axis = 0)
        TP_mn = np.mean(TP, axis = 0)
        AUC_mn= metrics.auc(TP_mn, FP_mn)
        
        print ("==============================\n")
        print FP_mn
        print ("==============================\n")

        line, = plt.plot(FP_mn, TP_mn, '-', label= 'WinLn: %03d, Feature: %03d, AUC = %f' % (window_length, feature_type, AUC_mn), linewidth=3 , color = colors[feature_type_idx])
        # Print the AUC
        print ("For Window Lenght %03d, feature type %03d, AUC = %f" % (window_length, feature_type, AUC_mn))

# naive classifier
print ("\nCreating curve for naive classifier ...")

comb_full = comb_train.append(comb_test)
targets = comb_full['Whisker']
predictions = [0] * len(targets)
predictions = SArray(predictions)
roc_curve =  gl.evaluation.roc_curve(targets, predictions)
FP = roc_curve['fpr']
TP = roc_curve['tpr']

plt.plot(FP, TP, '-.', label='Naive Classifier', linewidth=3)
  
# leg = plt.legend(loc='lower right')
# for legobj in leg.legendHandles:
#     legobj.set_linewidth(2.0)

plt.show() 


In [2]:
# load and organize feature matrix
def lz_load_feature_matrix_across_subject(mouse_id_test, window_length, window_step):
    # the mouse_id is the one that saved for the testing set
    # the mice other than the mouse_id are used for the training set
    # return combined train and testing data if "mouse_id == 0"
    
    mouse_id_full = range(1,7)
    mouse_id_trian = np.delete(mouse_id_full, mouse_id_test-1)
    mouse_id_train = list(mouse_id_trian)
    
    # load training dataset
    idx = 0
    for mouse_id_load in mouse_id_train:
        idx = idx + 1
        fileName = "format4ML_GC6f_emx_0" + str(mouse_id_load) + "_windowLen" + str(window_length) + "_winStep_0" + str(window_step) + "_v2_threshold_10.csv"
        loadPath = "/Users/lizhu/Dropbox/projects/calcium/format4ML/" + fileName

        comb = gl.SFrame.read_csv(loadPath, delimiter=',',header=False,verbose = False)
        colName_dg = 'degree'
        colName_dg = gl.SArray([colName_dg + repr(i+1) for i in range(30)])
        colName_cc = 'clusteringCoef'
        colName_cc = gl.SArray([colName_cc + repr(i+1) for i in range(30)])
        colName_pl = 'pathlength'
        colName_pl = gl.SArray([colName_pl + repr(i+1) for i in range(30)])
        colName = colName_dg.append(colName_cc.append(colName_pl.append(gl.SArray(['Whisker']))))
        colName = (list(colName))
        dictionary = dict(zip(comb.column_names(), colName))
        comb = comb.rename(dictionary)
        comb = gl.toolkits.cross_validation.shuffle(comb, random_seed=1)
        
        if idx == 1:
            comb_train = comb
        else:
            comb_train = comb_train.append(comb)
                    
    # load testing dataset
    if mouse_id_test == 0:
        mouse_id_test = 6
        
    fileName = "format4ML_GC6f_emx_0" + str(mouse_id_test) + "_windowLen" + str(window_length) + "_winStep_0" + str(window_step) + "_v2_threshold_10.csv"
    loadPath = "/Users/lizhu/Dropbox/projects/calcium/format4ML/" + fileName

    comb = gl.SFrame.read_csv(loadPath, delimiter=',',header=False,verbose = False)
    colName_dg = 'degree'
    colName_dg = gl.SArray([colName_dg + repr(i+1) for i in range(30)])
    colName_cc = 'clusteringCoef'
    colName_cc = gl.SArray([colName_cc + repr(i+1) for i in range(30)])
    colName_pl = 'pathlength'
    colName_pl = gl.SArray([colName_pl + repr(i+1) for i in range(30)])
    colName = colName_dg.append(colName_cc.append(colName_pl.append(gl.SArray(['Whisker']))))
    colName = (list(colName))
    dictionary = dict(zip(comb.column_names(), colName))
    comb = comb.rename(dictionary)
    comb_test = gl.toolkits.cross_validation.shuffle(comb, random_seed=1)
        
    if mouse_id_test != 0:
        return comb_train, comb_test
    else:
        comb_full = comb_train.append(comb_test)
        print type(comb_full)
        return comb_full

In [23]:
# train and cross-validation
def lz_KNN_across_subjects_AC_SE_SP(data_train, data_test, feature_type, k):
    
    # clearify features
    feature_dg = data_train.column_names()[0:30] # feature 1~30: degree, 31~60: clustering coefficient, 60~90: pathlength
    feature_cc = data_train.column_names()[30:60]
    feature_pl = data_train.column_names()[60:90]
    
    if feature_type == 1:   feature = feature_dg
    if feature_type == 2:   feature = feature_cc
    if feature_type == 3:   feature = feature_pl
    if feature_type == 12:  feature = feature_dg + feature_cc
    if feature_type == 13:  feature = feature_dg + feature_pl
    if feature_type == 23:  feature = feature_cc + feature_pl
    if feature_type == 123: feature = feature_dg + feature_cc + feature_pl
     
    m = gl.nearest_neighbor_classifier.create(data_train,
                                              target = 'Whisker',
                                              features = feature,
                                              distance = 'euclidean',
                                              verbose = False)

    results = m.evaluate(data_test, max_neighbors = k)
    
    return results

In [14]:
def lz_extract_ACC_SE_SP(confusion_matrix):
    TP = confusion_matrix[(confusion_matrix['target_label']==1) & (confusion_matrix['predicted_label']==1)]
    if np.size(TP) == 0:
        TP = 0
    else:
        TP = TP['count'][0]
    TN = confusion_matrix[(confusion_matrix['target_label']==0) & (confusion_matrix['predicted_label']==0)]
    if np.size(TN) == 0:
        TN = 0
    else:
        TN = TN['count'][0]
    FP = confusion_matrix[(confusion_matrix['target_label']==0) & (confusion_matrix['predicted_label']==1)]
    if np.size(FP) == 0:
        FP = 0
    else:
        FP = FP['count'][0]
    FN = confusion_matrix[(confusion_matrix['target_label']==1) & (confusion_matrix['predicted_label']==0)]
    if np.size(FN) == 0:
        FN = 0
    else:
        FN = FN['count'][0]
    
    return TP, TN, FP, FN